# Boilerplate


In [1]:

!pip install --upgrade https://github.com/arviz-devs/arviz/archive/v0.11.1.zip
!pip install --upgrade https://github.com/pymc-devs/pymc3/archive/v3.10.0.zip

import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import arviz as az

  Using cached https://github.com/arviz-devs/arviz/archive/v0.11.1.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for arviz: filename=arviz-0.11.1-cp37-none-any.whl size=1549418 sha256=38e971ac8fe20c360e6846a5b2c5e033b74d918c465d46e20257a864582f7b64
  Stored in directory: /tmp/pip-ephem-wheel-cache-m30pg5f3/wheels/58/bf/14/51f5f1138a3d2e58a512e4013080bf35db8748f2651ad38e81
Successfully built arviz
  Found existing installation: arviz 0.11.1
    Uninstalling arviz-0.11.1:
      Successfully uninstalled arviz-0.11.1
  Using cached https://github.com/pymc-devs/pymc3/archive/v3.10.0.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pymc3: filename=pymc3-3.10.0-cp37-none-any.whl size=1896907 sha256=8d2103782199667f8e2c95ca95be6dcc4415a4f75521a4c93d34386ead116828
  Stored in directory: /tmp/pip-e

In [2]:
%config InlineBackend.figure_format = 'retina'
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)
az.style.use("arviz-darkgrid")
az.rcParams["stats.hdi_prob"] = 0.89  # sets default credible interval used by arviz

# Q2 Models



In [3]:
congressData = pd.read_csv("/congress.csv", header=0)
congressData

,inc86,inc88,inc90,v86,v88,v90,v86_adj,v88_adj,v90_adj,recog86,recog88,recog90
0,1,1,1,0.745036,0.772443,0.714029,0.745036,0.772443,0.714029,0.453786,2.095207,-0.253805
1,1,1,1,0.673845,0.636182,0.597050,0.673845,0.636182,0.597050,0.602701,0.455774,1.527571
2,1,1,0,0.696457,0.664928,0.521043,0.696457,0.664928,0.521043,0.504943,0.656609,0.191874
3,-1,-1,-1,0.464590,0.273834,0.234377,0.464590,0.273834,0.234377,-0.502163,-1.829661,-1.819449
4,-1,-1,0,0.391095,0.263613,0.477439,0.391095,0.263613,0.477439,0.633357,-0.947548,0.720765
...,...,...,...,...,...,...,...,...,...,...,...,...
430,1,0,1,0.726371,0.763288,0.750401,0.726371,0.763288,0.750401,-0.715950,0.680629,-0.178116
431,-1,-1,-1,0.347979,0.291337,0.437871,0.347979,0.291337,0.437871,-1.353221,-1.439196,0.143544
432,-1,-1,-1,0.421110,0.373454,0.480790,0.421110,0.373454,0.480790,-1.266811,-0.971340,-0.588231
433,0,-1,0,0.387486,0.440967,0.607843,0.387486,0.440967,0.607843,-0.594891,-0.881916,0.076238


**2.1 Models**

In [4]:
with pm.Model() as incumbent :
    sigma = pm.Exponential("sigma", 1)
    alpha = pm.Normal('alpha', mu=0, sigma=0.2)
    beta86 = pm.Normal('beta86', mu=np.mean(congressData['v86_adj']), sigma=np.std(congressData['v86_adj']))
    betaInc88 = pm.Normal('betaInc88', mu=np.mean(congressData['inc88']), sigma=np.std(congressData['inc88']))
    inc_mus = pm.Deterministic('inc_mus', alpha + beta86 * congressData['v86_adj'] + betaInc88*congressData['inc88'])
    inc_votes = pm.Normal('inc_votes', mu=inc_mus, sigma=sigma, observed=congressData['v88_adj'])

    post_vote = pm.sample(1000, tune=1000)
    pred_vote = pm.sample_posterior_predictive(post_vote)
    incumbent_trace = pm.sample(return_inferencedata=True)

with pm.Model() as incumbent_no86 :
    sigma = pm.Exponential("sigma", 1)
    alpha = pm.Normal('alpha', mu=0, sigma=0.2)
    betaInc88 = pm.Normal('betaInc88', mu=np.mean(congressData['inc88']), sigma=np.std(congressData['inc88']))
    inc_no86_mus = pm.Deterministic('inc_no86_mus', alpha + betaInc88*congressData['inc88'])
    inc_no86_votes = pm.Normal('inc_no86_votes', mu=inc_no86_mus, sigma=sigma, observed=congressData['v88_adj'])

    post_no86 = pm.sample(1000, tune=1000)
    pred_no86 = pm.sample_posterior_predictive(post_no86)
    incumbent_no86_trace = pm.sample(return_inferencedata=True)



/usr/local/lib/python3.7/dist-packages/pymc3/sampling.py:468: FutureWarning: In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  FutureWarning,
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [betaInc88, beta86, alpha, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 6 seconds.
The number of effective samples is smaller than 25% for some parameters.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [betaInc88, beta86, alpha, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 6 seconds.
/usr/local/lib/python3.7/dist-packages/pymc3/sampling.py:468: FutureWarning: In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  FutureWarning,
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [betaInc88, alpha, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 3 seconds.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [betaInc88, alpha, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 3 seconds.


In [5]:
az.summary(post_vote, '~inc_mus')

/usr/local/lib/python3.7/dist-packages/arviz/data/io_pymc3.py:92: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
alpha,0.236,0.016,0.208,0.260,0.001,0.001,506.0,506.0,506.0,662.0,1.01
beta86,0.525,0.031,0.477,0.578,0.001,0.001,501.0,500.0,500.0,621.0,1.01
betaInc88,0.096,0.007,0.086,0.107,0.000,0.000,545.0,545.0,545.0,751.0,1.00
sigma,0.067,0.002,0.064,0.071,0.000,0.000,1003.0,1002.0,1001.0,1053.0,1.00


In [6]:
az.waic(incumbent_trace, incumbent)

/usr/local/lib/python3.7/dist-packages/arviz/stats/stats.py:1468: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  "For one or more samples the posterior variance of the log predictive "


Computed from 2000 by 435 log-likelihood matrix

          Estimate       SE
elpd_waic   553.93    19.74
p_waic        6.01        -

There has been a warning during the calculation. Please check the results.

In [7]:
az.summary(post_no86, '~inc_no86_mus')

/usr/local/lib/python3.7/dist-packages/arviz/data/io_pymc3.py:92: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
alpha,0.506,0.004,0.499,0.512,0.0,0.0,2585.0,2585.0,2558.0,1415.0,1.0
betaInc88,0.191,0.004,0.184,0.197,0.0,0.0,2818.0,2799.0,2818.0,1158.0,1.0
sigma,0.085,0.003,0.080,0.089,0.0,0.0,2081.0,2051.0,2125.0,1466.0,1.0


In [8]:
az.waic(incumbent_no86_trace, incumbent_no86)

Computed from 2000 by 435 log-likelihood matrix

          Estimate       SE
elpd_waic   452.99    18.27
p_waic        3.38        -

**2.2 Model**

In [9]:
with pm.Model() as incumbent_90 :
    sigma = pm.Exponential("sigma", 1)
    alpha = pm.Normal('alpha', mu=0, sigma=0.2)
    beta88 = pm.Normal('beta88', mu=0.5, sigma=0.2)
    betaInc90 = pm.Normal('betaInc90', mu=np.mean(congressData['inc90']), sigma=np.std(congressData['inc90']))
    inc_90_mus = pm.Deterministic('inc_90_mus', alpha + beta88*inc_votes + betaInc90*congressData['inc90'])
    votes_90 = pm.Normal('votes_90', mu=inc_90_mus, sigma=sigma, observed=congressData['v90_adj'])

    post_90 = pm.sample(1000, tune=1000)
    pred_90 = pm.sample_posterior_predictive(post_90)
    incumbent_90_trace = pm.sample(return_inferencedata=True)

/usr/local/lib/python3.7/dist-packages/pymc3/sampling.py:468: FutureWarning: In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  FutureWarning,
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [betaInc90, beta88, alpha, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 6 seconds.
The number of effective samples is smaller than 25% for some parameters.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [betaInc90, beta88, alpha, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 6 seconds.
The acceptance probability does not match the target. It is 0.8890663533488369, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.


In [10]:
az.summary(post_90, '~inc_90_mus')

/usr/local/lib/python3.7/dist-packages/arviz/data/io_pymc3.py:92: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
alpha,0.250,0.020,0.218,0.280,0.001,0.001,504.0,501.0,503.0,657.0,1.0
beta88,0.519,0.039,0.457,0.580,0.002,0.001,492.0,492.0,491.0,652.0,1.0
betaInc90,0.064,0.008,0.051,0.077,0.000,0.000,552.0,533.0,552.0,693.0,1.0
sigma,0.081,0.003,0.077,0.085,0.000,0.000,1057.0,1054.0,1056.0,1042.0,1.0


In [11]:
az.waic(incumbent_90_trace, incumbent_90)

/usr/local/lib/python3.7/dist-packages/arviz/stats/stats.py:1468: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  "For one or more samples the posterior variance of the log predictive "


Computed from 2000 by 435 log-likelihood matrix

          Estimate       SE
elpd_waic   477.15    19.17
p_waic        5.08        -

There has been a warning during the calculation. Please check the results.

In [12]:
az.plot_trace(incumbent_90_trace)

Output hidden; open in https://colab.research.google.com to view.

HDI per Seat

In [67]:
az.hdi(pred_90['votes_90'], hdi_prob=0.89)

/usr/local/lib/python3.7/dist-packages/arviz/stats/stats.py:496: FutureWarning: hdi currently interprets 2d data as (draw, shape) but this will change in a future release to (chain, draw) for coherence with other functions
  FutureWarning,


array([[ 2.42904646e-02,  1.62205766e+00],
       [-1.19392286e-02,  1.58819433e+00],
       [-6.71112081e-02,  1.55186031e+00],
       [-4.23700684e-01,  1.14857194e+00],
       [-4.17905408e-01,  1.17100634e+00],
       [-4.97139857e-01,  1.11093094e+00],
       [-8.37390974e-02,  1.49841401e+00],
       [-5.69520934e-01,  1.02944904e+00],
       [-5.59643180e-01,  1.01738980e+00],
       [-5.34799575e-02,  1.58332771e+00],
       [-2.55047549e-02,  1.57666996e+00],
       [ 3.71896561e-02,  1.64589893e+00],
       [ 1.73137173e-02,  1.65105797e+00],
       [ 3.20432295e-02,  1.66478916e+00],
       [-1.67557173e-02,  1.58613561e+00],
       [-3.01040971e-02,  1.59073406e+00],
       [-2.86421051e-02,  1.56479478e+00],
       [-1.75013915e-02,  1.57350452e+00],
       [ 5.99565416e-02,  1.69478958e+00],
       [-3.73523281e-01,  1.20717353e+00],
       [-4.98155402e-01,  1.08800814e+00],
       [-1.51166906e-01,  1.45207071e+00],
       [-4.60928263e-01,  1.10959224e+00],
       [ 2.

True Vote Results

In [53]:
v90 = np.array(congressData['v90'])
v90Dem = v90[np.where(v90 > 0.5)]
np.shape(v90Dem)

(267,)

Simulated Results

In [87]:
estimated_v90 = np.array([np.mean(arr) for arr in np.array(az.hdi(pred_90['votes_90'], hdi_prob=0.89))])
np.shape(np.where(estimated_v90 > 0.5))

/usr/local/lib/python3.7/dist-packages/arviz/stats/stats.py:496: FutureWarning: hdi currently interprets 2d data as (draw, shape) but this will change in a future release to (chain, draw) for coherence with other functions
  FutureWarning,


(1, 260)

**2.3**

In [90]:
with pm.Model() as incumbent_recog :
    sigma = pm.Exponential("sigma", 1)
    alpha = pm.Normal('alpha', mu=0, sigma=0.2)
    beta86 = pm.Normal('beta86', mu=np.mean(congressData['v86_adj']), sigma=np.std(congressData['v86_adj']))
    betaInc88 = pm.Normal('betaInc88', mu=np.mean(congressData['inc88']), sigma=np.std(congressData['inc88']))
    betaRecog88 = pm.Normal('betaRecog88', mu=np.mean(congressData['recog88']), sigma=np.std(congressData['recog88']))
    inc_recog_mus = pm.Deterministic('inc_recog_mus', alpha + beta86 * congressData['v86_adj'] + betaInc88*congressData['inc88'] + betaRecog88*congressData['recog88'])
    inc_recog_votes = pm.Normal('inc_recog_votes', mu=inc_recog_mus, sigma=sigma, observed=congressData['v88_adj'])

    post_vote_recog = pm.sample(1000, tune=1000)
    pred_vote_recog = pm.sample_posterior_predictive(post_vote_recog)
    incumbent_recog_trace = pm.sample(return_inferencedata=True)

with pm.Model() as incumbent_recogsq :
    sigma = pm.Exponential("sigma", 1)
    alpha = pm.Normal('alpha', mu=0, sigma=0.2)
    beta86 = pm.Normal('beta86', mu=np.mean(congressData['v86_adj']), sigma=np.std(congressData['v86_adj']))
    betaInc88 = pm.Normal('betaInc88', mu=np.mean(congressData['inc88']), sigma=np.std(congressData['inc88']))
    betaRecog88 = pm.Normal('betaRecog88', mu=np.mean(congressData['recog88']), sigma=np.std(congressData['recog88']))
    betaRecog88sq = pm.Normal('betaRecog88sq', mu=np.mean(congressData['recog88']), sigma=np.std(congressData['recog88']))
    inc_recogsq_mus = pm.Deterministic('inc_recogsq_mus', alpha + beta86 * congressData['v86_adj'] + betaInc88*congressData['inc88'] + betaRecog88*congressData['recog88'] + betaRecog88sq*congressData['recog88']**2)
    inc_recogsq_votes = pm.Normal('inc_recogsq_votes', mu=inc_recogsq_mus, sigma=sigma, observed=congressData['v88_adj'])

    post_vote_recogsq = pm.sample(1000, tune=1000)
    pred_vote_recogsq = pm.sample_posterior_predictive(post_vote_recogsq)
    incumbent_recogsq_trace = pm.sample(return_inferencedata=True)

/usr/local/lib/python3.7/dist-packages/pymc3/sampling.py:468: FutureWarning: In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  FutureWarning,
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [betaRecog88, betaInc88, beta86, alpha, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 7 seconds.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [betaRecog88, betaInc88, beta86, alpha, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 7 seconds.
The acceptance probability does not match the target. It is 0.7216308307148386, but should be close to 0.8. Try to increase the number of tuning steps.
/usr/local/lib/python3.7/dist-packages/pymc3/sampling.py:468: FutureWarning: In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  FutureWarning,
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [betaRecog88sq, betaRecog88, betaInc88, beta86, alpha, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 9 seconds.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [betaRecog88sq, betaRecog88, betaInc88, beta86, alpha, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 9 seconds.


In [91]:
az.summary(post_vote_recog, '~inc_recog_mus')

/usr/local/lib/python3.7/dist-packages/arviz/data/io_pymc3.py:92: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
alpha,0.259,0.017,0.233,0.286,0.001,0.000,716.0,716.0,715.0,960.0,1.0
beta86,0.488,0.032,0.436,0.538,0.001,0.001,711.0,711.0,708.0,1001.0,1.0
betaInc88,0.083,0.007,0.072,0.094,0.000,0.000,771.0,761.0,775.0,979.0,1.0
betaRecog88,0.026,0.005,0.019,0.033,0.000,0.000,880.0,880.0,881.0,839.0,1.0
sigma,0.065,0.002,0.061,0.068,0.000,0.000,1127.0,1127.0,1113.0,850.0,1.0


In [92]:
az.waic(incumbent_recog_trace, inc_recog_votes)

/usr/local/lib/python3.7/dist-packages/arviz/stats/stats.py:1468: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  "For one or more samples the posterior variance of the log predictive "


Computed from 2000 by 435 log-likelihood matrix

          Estimate       SE
elpd_waic   569.10    18.12
p_waic        6.81        -

There has been a warning during the calculation. Please check the results.

In [96]:
az.summary(post_vote_recogsq, '~inc_recogsq_mus')

/usr/local/lib/python3.7/dist-packages/arviz/data/io_pymc3.py:92: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
alpha,0.263,0.016,0.237,0.289,0.001,0.000,923.0,918.0,925.0,1053.0,1.0
beta86,0.487,0.030,0.442,0.536,0.001,0.001,894.0,894.0,897.0,922.0,1.0
betaInc88,0.082,0.007,0.072,0.093,0.000,0.000,1063.0,1063.0,1062.0,1370.0,1.0
betaRecog88,0.025,0.005,0.018,0.033,0.000,0.000,1588.0,1588.0,1590.0,1400.0,1.0
betaRecog88sq,-0.003,0.003,-0.008,0.000,0.000,0.000,1852.0,1452.0,1848.0,1267.0,1.0
sigma,0.065,0.002,0.062,0.069,0.000,0.000,1346.0,1341.0,1344.0,1191.0,1.0


In [94]:
az.waic(incumbent_recogsq_trace, inc_recogsq_votes)

/usr/local/lib/python3.7/dist-packages/arviz/stats/stats.py:1468: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  "For one or more samples the posterior variance of the log predictive "


Computed from 2000 by 435 log-likelihood matrix

          Estimate       SE
elpd_waic   569.54    18.11
p_waic        7.16        -

There has been a warning during the calculation. Please check the results.

In [95]:
compare_df = az.compare(
    {
        "incumbent+vote": incumbent_trace,
        "incumbent+vote+recog": incumbent_recog_trace,
        "incumbent+vote+recog+recogsq": incumbent_recogsq_trace,
    }
)
compare_df

/usr/local/lib/python3.7/dist-packages/arviz/stats/stats.py:147: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
incumbent+vote+recog+recogsq,0,569.557546,7.140604,0.000000,0.792137,18.100218,0.000000,False,log
incumbent+vote+recog,1,569.043487,6.864367,0.514059,0.207863,18.148028,1.328016,False,log
incumbent+vote,2,553.921097,6.023377,15.636449,0.000000,19.745824,5.606122,False,log
